# Import Requirements

In [3]:
from __future__ import division
import glob
import pandas as pd
import numpy as np
import itertools
from collections import Counter
import time
import matplotlib.pyplot as plt
%matplotlib inline

# Import data

In [7]:
DF = pd.read_csv('/Users/greg/Desktop/FullNeuroIllumina/complete_data.csv')
print DF.shape
DF.head()

(3900, 26)


,Patient,Visit,GDS,Prot,AAPos,Coverage,A,R,N,D,...,L,K,M,F,P,S,T,W,Y,V
0,A0001,R09,0.583333,Tat1,1,38938,0.000051,0.002157,0.000051,0.000077,...,0.006934,0.001695,0.565052,0.000026,0,0.000848,0.000128,0.416637,0,0.000205
1,A0010,R08,1.416667,Tat1,1,171,0.000000,0.000000,0.000000,0.000000,...,0.005848,0.005848,0.584795,0.000000,0,0.000000,0.000000,0.403509,0,0.000000
2,A0013,R09,0.000000,Tat1,1,602,0.000000,0.001661,0.000000,0.000000,...,0.001661,0.004983,0.282392,0.000000,0,0.001661,0.000000,0.209302,0,0.000000
3,A0019,R12,0.250000,Tat1,1,1485,0.000000,0.001347,0.000673,0.000000,...,0.008081,0.003367,0.526599,0.000000,0,0.001347,0.000673,0.452525,0,0.000000
4,A0026,R09,0.583333,Tat1,1,334,0.000000,0.002994,0.000000,0.000000,...,0.008982,0.000000,0.589820,0.000000,0,0.002994,0.000000,0.389222,0,0.000000


# Functions

In [5]:
def slice_position(df, pos):
    m = df['AAPos'] == pos
    sliced_df = df[m]
    return sliced_df

def reformat_section(df):
    '''Must be a dataframe containing only 1 position'''
    pos = str(int(list(df.AAPos)[0]))
    AAs = ['A','R','N','D','C','Q','E','G','H','I',
           'L','K','M','F','P','S','T','W','Y','V']
    newnames_dict = {}
    for item in AAs:
        newnames_dict[item]= pos+item
    df_renamed = df.rename(columns = newnames_dict)
    df_renamed.drop(['AAPos','Coverage'], axis=1, inplace=True)
    return df_renamed

# Reformat

In [9]:
tat1_length = 72

pos1_df = slice_position(DF, 1)
DF2 = reformat_section(pos1_df)
for i in range(2,tat1_length+1):
    pos_df = slice_position(DF, i)
    pos_df = reformat_section(pos_df)
    DF2 = pd.merge(DF2, pos_df,
               left_on=['Patient','Visit','GDS','Prot',],
               right_on=['Patient','Visit','GDS','Prot'],
               how='outer')
    
print DF2.shape
DF2.head()

(55, 1444)


,Patient,Visit,GDS,Prot,1A,1R,1N,1D,1C,1Q,...,72L,72K,72M,72F,72P,72S,72T,72W,72Y,72V
0,A0001,R09,0.583333,Tat1,0.000051,0.002157,0.000051,0.000077,0.002080,0.000051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0010,R08,1.416667,Tat1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A0013,R09,0.000000,Tat1,0.000000,0.001661,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A0019,R12,0.250000,Tat1,0.000000,0.001347,0.000673,0.000000,0.000673,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A0026,R09,0.583333,Tat1,0.000000,0.002994,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
DF2.fillna(DF2.mean(), inplace=True)
DF2['GDS_Threshold'] = DF2['GDS'] > 0.5
DF2['GDS_Threshold'] = DF2['GDS_Threshold'].astype(int)
print DF2.shape
DF2.head()

(55, 1445)


,Patient,Visit,GDS,Prot,1A,1R,1N,1D,1C,1Q,...,72K,72M,72F,72P,72S,72T,72W,72Y,72V,GDS_Threshold
0,A0001,R09,0.583333,Tat1,0.000051,0.002157,0.000051,0.000077,0.002080,0.000051,...,0.049947,0.000117,0.000115,0.115569,0.00037,0.000166,0,0.000057,0.000062,1
1,A0010,R08,1.416667,Tat1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.049947,0.000117,0.000115,0.115569,0.00037,0.000166,0,0.000057,0.000062,1
2,A0013,R09,0.000000,Tat1,0.000000,0.001661,0.000000,0.000000,0.000000,0.000000,...,0.049947,0.000117,0.000115,0.115569,0.00037,0.000166,0,0.000057,0.000062,0
3,A0019,R12,0.250000,Tat1,0.000000,0.001347,0.000673,0.000000,0.000673,0.000000,...,0.049947,0.000117,0.000115,0.115569,0.00037,0.000166,0,0.000057,0.000062,0
4,A0026,R09,0.583333,Tat1,0.000000,0.002994,0.000000,0.000000,0.000000,0.000000,...,0.049947,0.000117,0.000115,0.115569,0.00037,0.000166,0,0.000057,0.000062,1


# Machine Learning approach (classification)

In [82]:
from sklearn.feature_selection import VarianceThreshold
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA

# Data import

In [83]:
y = np.ravel(DF2['GDS_Threshold'])
X = DF2.drop(['Patient','Visit','GDS','Prot','GDS_Threshold'], axis=1)
X = X.as_matrix()
print X.shape
print y.shape

(55, 1440)
(55,)


# Feature Selection (variance threshold)

In [84]:
for i in range(20):
    try:
        selector = VarianceThreshold(threshold=i/50)
        print i/50, selector.fit_transform(X).shape
        #print X.head()
    except ValueError:
        continue    

0.0 (55, 1308)
0.02 (55, 113)
0.04 (55, 78)
0.06 (55, 61)
0.08 (55, 45)
0.1 (55, 40)
0.12 (55, 29)
0.14 (55, 25)
0.16 (55, 19)
0.18 (55, 13)
0.2 (55, 9)
0.22 (55, 2)
0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38


In [85]:
selector = VarianceThreshold()
X = selector.fit_transform(X)
X.shape

(55, 1308)

# Test Train Split

In [86]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.4, random_state=0)

print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(33, 1308) (33,)
(22, 1308) (22,)


# Scoring

In [90]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

NameError: name 'LinearDiscriminantAnalysis' is not defined

In [95]:
clf1 = SVC(kernel='linear', C=1).fit(X_train, y_train)
clf2 = KNeighborsClassifier(3).fit(X_train, y_train)
clf3 = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
clf4 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1).fit(X_train, y_train)
clf5 = KNeighborsClassifier(3).fit(X_train, y_train)
clf6 = AdaBoostClassifier().fit(X_train, y_train)
clf7 = GaussianNB().fit(X_train, y_train)
clf8 = LDA().fit(X_train, y_train)
clf9 = QDA().fit(X_train, y_train)

0.40909090909090912

In [96]:
# It will use 5+ different classification algorithms
# It will use various test/train splits
# It will rank feature importance
# It will compare with clinical data